<a href="https://colab.research.google.com/github/Shrxxvxthsx/Personal-Financial-Dashboard/blob/main/Notebooks/finance_dashboard_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 Personal Finance Dashboard — Google Colab Notebook

This notebook downloads a sample dataset from Kaggle, cleans & categorizes transactions, builds monthly summaries, and exports CSVs ready for Power BI/Tableau.


In [ ]:
# --- STEP 1: Install and Configure Kaggle ---
!pip install kaggle --quiet
!mkdir -p ~/.kaggle
from google.colab import files
print('📂 Please upload your kaggle.json file (Kaggle > Account > Create API Token)')
files.upload()  # Upload your Kaggle API key file
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# --- STEP 2: Download Dataset from Kaggle ---
!kaggle datasets download -d ismetsemedov/personal-budget-transactions-dataset -p /content
!unzip -o /content/personal-budget-transactions-dataset.zip -d /content


In [ ]:
# --- STEP 3: Load and Inspect Dataset ---
import pandas as pd
import os

path = '/content/Personal Budget Transactions.csv'
df = pd.read_csv(path)
print('✅ Data loaded. Shape:', df.shape)
df.head()


In [ ]:
# --- STEP 4: Clean and Categorize ---
df.columns = df.columns.str.strip().str.lower()
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['description'] = df['description'].astype(str).str.strip()

def categorize(desc: str) -> str:
    d = desc.lower()
    if 'salary' in d or 'income' in d:
        return 'Income'
    if 'amazon' in d or 'flipkart' in d:
        return 'Shopping'
    if 'uber' in d or 'ola' in d:
        return 'Transport'
    if 'zomato' in d or 'swiggy' in d:
        return 'Food & Dining'
    if 'electricity' in d or 'water' in d or 'gas' in d:
        return 'Utilities'
    if 'rent' in d:
        return 'Housing'
    return 'Other'

df['category'] = df['description'].apply(categorize)
df = df.sort_values('date')

print('🧹 Cleaning complete. Columns:', list(df.columns))
df.head()


In [ ]:
# --- STEP 5: Save Cleaned Data ---
clean_path = '/content/transactions_cleaned.csv'
df.to_csv(clean_path, index=False)
print('💾 Saved', clean_path)


In [ ]:
# --- STEP 6: Monthly Summary ---
df['year_month'] = df['date'].dt.to_period('M')
monthly_summary = df.groupby(['year_month', 'category'])['amount'].sum().reset_index()
monthly_pivot = monthly_summary.pivot(index='year_month', columns='category', values='amount').fillna(0).reset_index()

summary_path = '/content/monthly_summary.csv'
monthly_pivot.to_csv(summary_path, index=False)
print('💾 Saved', summary_path)
monthly_pivot.head()


In [ ]:
# --- STEP 7: Quick Preview Charts (matplotlib only) ---
import matplotlib.pyplot as plt

# Convert period to string for plotting
x = monthly_pivot['year_month'].astype(str)

# Income vs Total Expenses
income = monthly_pivot['Income'] if 'Income' in monthly_pivot.columns else None
expense_cols = [c for c in monthly_pivot.columns if c not in ['year_month', 'Income']]
total_expenses = monthly_pivot[expense_cols].sum(axis=1) if expense_cols else None

plt.figure(figsize=(10,5))
if income is not None:
    plt.plot(x, income, marker='o', label='Income')
if total_expenses is not None:
    plt.plot(x, total_expenses, marker='o', label='Total Expenses')
plt.title('Monthly Income vs Expenses')
plt.xlabel('Month')
plt.ylabel('Amount')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# Expense Breakdown Pie (on full dataset)
expense_mask = df['amount'] < 0
if expense_mask.any():
    expense_series = df.loc[expense_mask].groupby('category')['amount'].sum().abs()
    plt.figure(figsize=(6,6))
    plt.pie(expense_series.values, labels=expense_series.index, autopct='%1.1f%%', startangle=90)
    plt.title('Expense Breakdown by Category')
    plt.tight_layout()
    plt.show()


In [ ]:
# --- STEP 8: Copy outputs to /content/data for easy download ---
import shutil, os
os.makedirs('/content/data', exist_ok=True)
shutil.copy('/content/transactions_cleaned.csv', '/content/data/transactions_cleaned.csv')
shutil.copy('/content/monthly_summary.csv', '/content/data/monthly_summary.csv')
print('📦 Outputs available in /content/data/')
